In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Chronic_kidney_disease"
cohort = "GSE104954"

# Input paths
in_trait_dir = "../../input/GEO/Chronic_kidney_disease"
in_cohort_dir = "../../input/GEO/Chronic_kidney_disease/GSE104954"

# Output paths
out_data_file = "../../output/preprocess/Chronic_kidney_disease/GSE104954.csv"
out_gene_data_file = "../../output/preprocess/Chronic_kidney_disease/gene_data/GSE104954.csv"
out_clinical_data_file = "../../output/preprocess/Chronic_kidney_disease/clinical_data/GSE104954.csv"
json_path = "../../output/preprocess/Chronic_kidney_disease/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# 1. Analyze gene expression availability
import numpy as np
import pandas as pd
import os

is_gene_available = True  # Based on the background information mentioning "transcriptome" and "hybridization on Affymetrix microarrays"

# 2. Variable availability and data type conversion
# 2.1 Identify rows in sample characteristics dictionary for each variable
trait_row = 1  # diagnosis is in row 1
age_row = None  # age not available in the data
gender_row = None  # gender not available in the data

# 2.2 Define conversion functions for each variable
def convert_trait(value):
    """Convert diagnosis values to binary indicating chronic kidney disease status."""
    if pd.isna(value):
        return None
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # All diagnoses in the dataset represent forms of chronic kidney disease
    # except possibly "Tumor nephrectomy" which is a procedure
    if value == "Tumor nephrectomy":
        return 0  # Not CKD
    else:
        return 1  # CKD condition
    
def convert_age(value):
    """Placeholder function for age conversion."""
    return None  # Age data not available

def convert_gender(value):
    """Placeholder function for gender conversion."""
    return None  # Gender data not available

# 3. Save metadata about dataset usability
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. If trait data is available, extract clinical features
if trait_row is not None:
    # Create a proper clinical data DataFrame from the sample characteristics dictionary
    # Using a format that matches what geo_select_clinical_features expects
    
    # Initialize an empty dataframe
    clinical_data = pd.DataFrame()
    
    # Add the sample characteristics as columns
    clinical_data[0] = ['tissue: Tubulointerstitium from kidney biopsy']
    clinical_data[1] = ['diagnosis: Diabetic nephropathy']  # We'll add one value and update later
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the data
    preview = preview_df(selected_clinical_df)
    print("Clinical data preview:", preview)
    
    # Save clinical data to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


### Step 3: Dataset Analysis and Clinical Feature Extraction

In [ ]:
import os
import pandas as pd
import numpy as np
import json
from typing import Callable, Optional, Dict, Any

# First, we need to load the needed data
# Let's assume the clinical_data DataFrame was already loaded in a previous step
# If not available, we need to load it first
try:
    clinical_data
except NameError:
    # Load the clinical data if not already loaded
    clinical_data_path = os.path.join(in_cohort_dir, "clinical_data.csv")
    if os.path.exists(clinical_data_path):
        clinical_data = pd.read_csv(clinical_data_path)
    else:
        # Try alternative location
        clinical_data_path = os.path.join(in_cohort_dir, "sample_characteristics.csv")
        if os.path.exists(clinical_data_path):
            clinical_data = pd.read_csv(clinical_data_path)
        else:
            raise FileNotFoundError(f"Clinical data file not found at {clinical_data_path}")

# Check if we have gene expression data (not miRNA or methylation)
# This requires examining the available data files
gene_files = [f for f in os.listdir(in_cohort_dir) if f.endswith('.txt') or f.endswith('.csv') or f.endswith('.tsv')]
gene_expression_patterns = ['expr', 'gene', 'rna', 'expression']
has_gene_files = any(any(pattern in f.lower() for pattern in gene_expression_patterns) for f in gene_files)

is_gene_available = has_gene_files  # Set based on file examination
if not is_gene_available:
    # If we couldn't find evidence from filenames, let's check if we have any matrix files that might contain gene data
    matrix_files = [f for f in os.listdir(in_cohort_dir) if 'matrix' in f.lower()]
    is_gene_available = len(matrix_files) > 0

# Inspect the clinical data to understand what's available
print("Clinical data columns:", clinical_data.columns.tolist())
print("Clinical data shape:", clinical_data.shape)
print("First few rows of clinical data:")
print(clinical_data.head())

# Let's examine unique values in each row to identify relevant rows
unique_values = {}
for i in range(len(clinical_data)):
    row_values = clinical_data.iloc[i, 1:].unique()
    if len(row_values) > 1:  # Only consider rows with multiple values
        print(f"Row {i}: {clinical_data.iloc[i, 0]} - Unique values: {row_values}")
        unique_values[i] = row_values

# Based on the examination, determine key rows for trait, age, and gender
# For CKD, we're looking for rows related to kidney disease status, patient age, and gender/sex

# For trait (CKD), look for keywords like "disease", "status", "CKD", "kidney", etc.
trait_keywords = ["kidney", "ckd", "disease", "status", "diagnosis", "patient", "healthy", "control"]
trait_row = None
for i, values in unique_values.items():
    row_name = str(clinical_data.iloc[i, 0]).lower()
    if any(keyword in row_name for keyword in trait_keywords):
        if len(unique_values[i]) > 1:  # Ensure it's not a constant feature
            trait_row = i
            print(f"Trait row identified: {i} - {clinical_data.iloc[i, 0]}")
            break

# For age, look for "age" in the row name
age_row = None
for i, values in unique_values.items():
    row_name = str(clinical_data.iloc[i, 0]).lower()
    if "age" in row_name:
        if len(unique_values[i]) > 1:  # Ensure it's not a constant feature
            age_row = i
            print(f"Age row identified: {i} - {clinical_data.iloc[i, 0]}")
            break

# For gender, look for "gender", "sex", "male", "female" in the row name
gender_row = None
gender_keywords = ["gender", "sex", "male", "female"]
for i, values in unique_values.items():
    row_name = str(clinical_data.iloc[i, 0]).lower()
    if any(keyword in row_name for keyword in gender_keywords):
        if len(unique_values[i]) > 1:  # Ensure it's not a constant feature
            gender_row = i
            print(f"Gender row identified: {i} - {clinical_data.iloc[i, 0]}")
            break

# Define conversion functions for each variable
def convert_trait(value):
    if value is None or pd.isna(value):
        return None
    
    value = str(value).lower()
    # Extract value after colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Convert to binary based on common CKD terminology
    if any(term in value for term in ["ckd", "chronic kidney disease", "patient", "disease", "positive", "yes"]):
        return 1
    elif any(term in value for term in ["control", "healthy", "normal", "negative", "no"]):
        return 0
    else:
        return None

def convert_age(value):
    if value is None or pd.isna(value):
        return None
    
    value = str(value)
    # Extract value after colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Try to extract age value
    import re
    age_match = re.search(r'(\d+)', value)
    if age_match:
        return float(age_match.group(1))
    else:
        return None

def convert_gender(value):
    if value is None or pd.isna(value):
        return None
    
    value = str(value).lower()
    # Extract value after colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Convert to binary: female = 0, male = 1
    if any(term in value for term in ["female", "f", "woman", "women"]):
        return 0
    elif any(term in value for term in ["male", "m", "man", "men"]):
        return 1
    else:
        return None

# Determine if trait data is available
is_trait_available = trait_row is not None

# Save initial metadata
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Extract clinical features if trait data is available
if is_trait_available:
    # Extract features using the library function
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted data
    print("Preview of extracted clinical features:")
    print(preview_df(selected_clinical_df.T))
    
    # Save the extracted clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.T.to_csv(out_clinical_data_file, index=True)
    print(f"Clinical data saved to {out_clinical_data_file}")


### Step 4: Gene Data Extraction

In [ ]:
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# Set gene availability flag
is_gene_available = True  # Initially assume gene data is available

# First check if the matrix file contains the expected marker
found_marker = False
try:
    with gzip.open(matrix_file, 'rt') as file:
        for line in file:
            if "!series_matrix_table_begin" in line:
                found_marker = True
                break
    
    if found_marker:
        print("Found the matrix table marker in the file.")
    else:
        print("Warning: Could not find '!series_matrix_table_begin' marker in the file.")
        
    # Try to extract gene data from the matrix file
    gene_data = get_genetic_data(matrix_file)
    
    if gene_data.shape[0] == 0:
        print("Warning: Extracted gene data has 0 rows.")
        is_gene_available = False
    else:
        print(f"Gene data shape: {gene_data.shape}")
        # Print the first 20 gene/probe identifiers
        print("First 20 gene/probe identifiers:")
        print(gene_data.index[:20].tolist())
        
except Exception as e:
    print(f"Error extracting gene data: {e}")
    is_gene_available = False
    
    # Try to diagnose the file format
    print("Examining file content to diagnose the issue:")
    try:
        with gzip.open(matrix_file, 'rt') as file:
            for i, line in enumerate(file):
                if i < 10:  # Print first 10 lines to diagnose
                    print(f"Line {i}: {line.strip()[:100]}...")  # Print first 100 chars of each line
                else:
                    break
    except Exception as e2:
        print(f"Error examining file: {e2}")

if not is_gene_available:
    print("Gene expression data could not be successfully extracted from this dataset.")


### Step 5: Gene Identifier Review

In [ ]:
# Reviewing gene identifiers 
# The pattern "10000_at", "10001_at" suggests these are probe IDs from an Affymetrix microarray
# These are not standard human gene symbols and will need to be mapped to gene symbols

requires_gene_mapping = True


### Step 6: Gene Annotation

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Get a more complete view to understand the annotation structure
print("\nComplete sample of a few rows:")
print(gene_annotation.iloc[:3].to_string())

# Check if there are any columns that might contain gene information beyond what we've seen
potential_gene_columns = [col for col in gene_annotation.columns if 
                          any(term in col.upper() for term in ["GENE", "SYMBOL", "NAME", "ID"])]
print(f"\nPotential gene-related columns: {potential_gene_columns}")

# Look for additional columns that might contain gene symbols
# Since we only have 'ID' and 'ENTREZ_GENE_ID', check if we need to use Entrez IDs for mapping
gene_id_col = 'ID'
gene_symbol_col = None

# Check various potential column names for gene symbols
for col_name in ['GENE_SYMBOL', 'SYMBOL', 'GENE', 'GENE_NAME', 'GB_ACC']:
    if col_name in gene_annotation.columns:
        gene_symbol_col = col_name
        break

# If no dedicated symbol column is found, we'll need to use ENTREZ_GENE_ID
if gene_symbol_col is None and 'ENTREZ_GENE_ID' in gene_annotation.columns:
    gene_symbol_col = 'ENTREZ_GENE_ID'
    print("\nNo direct gene symbol column found. Will use Entrez Gene IDs for mapping.")

if gene_id_col in gene_annotation.columns and gene_symbol_col is not None:
    print(f"\nSample mappings from '{gene_id_col}' to '{gene_symbol_col}':")
    sample_mappings = gene_annotation[[gene_id_col, gene_symbol_col]].head(10)
    print(sample_mappings)
    
    # Check for non-null mappings to confirm data quality
    non_null_mappings = gene_annotation[[gene_id_col, gene_symbol_col]].dropna(subset=[gene_symbol_col])
    print(f"\nNumber of probes with gene ID mappings: {len(non_null_mappings)}")
    print(f"Sample of valid mappings:")
    print(non_null_mappings.head(5))
else:
    print("Required mapping columns not found in the annotation data. Will need to explore alternative mapping approaches.")


### Step 7: Gene Identifier Mapping

In [ ]:
# 1. Determine which columns contain the gene identifiers and gene symbols
gene_id_col = 'ID'  # This matches the identifiers seen in gene_data.index
gene_symbol_col = 'Symbol'  # This contains the human gene symbols

# 2. Create gene mapping dataframe using the identified columns
mapping_df = get_gene_mapping(gene_annotation, gene_id_col, gene_symbol_col)
print(f"Created mapping dataframe with shape: {mapping_df.shape}")
print("Sample of mapping data:")
print(mapping_df.head())

# 3. Apply gene mapping to convert probe-level measurements to gene-level expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"Converted gene expression data shape: {gene_data.shape}")
print("First few gene symbols after mapping:")
print(gene_data.index[:10].tolist())

# Check for genes with highest expression to verify successful mapping
print("\nSample of genes with highest mean expression:")
mean_expr = gene_data.mean(axis=1)
high_expr_genes = mean_expr.nlargest(5)
print(high_expr_genes)


### Step 8: Data Normalization and Linking

In [ ]:
# 1. First, load the genetic data from the matrix file again
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_data = get_genetic_data(matrix_file)
print(f"Loaded gene data shape: {gene_data.shape}")

# Get the gene annotation data for mapping
gene_annotation = get_gene_annotation(soft_file)
gene_id_col = 'ID'
gene_symbol_col = 'Symbol'
mapping_df = get_gene_mapping(gene_annotation, gene_id_col, gene_symbol_col)
print(f"Created mapping dataframe with shape: {mapping_df.shape}")

# Apply gene mapping to convert probe-level measurements to gene-level expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"Converted gene expression data shape: {gene_data.shape}")

# Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Extract trait information from the clinical_data
# Re-extract the clinical data from the matrix file
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# Get unique values from clinical data to understand the structure
sample_characteristics_dict = get_unique_values_by_row(clinical_data)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)

# Define the trait row and conversion function
trait_row = 1  # diagnosis is in row 1
def convert_trait(value):
    """Convert diagnosis values to binary indicating chronic kidney disease status."""
    if pd.isna(value):
        return None
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # All diagnoses in the dataset represent forms of chronic kidney disease
    # except possibly "Tumor nephrectomy" which is a procedure
    if value == "Tumor nephrectomy":
        return 0  # Not CKD
    else:
        return 1  # CKD condition

# Create the clinical dataframe
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=None,  # No age data
    convert_age=None,
    gender_row=None,  # No gender data
    convert_gender=None
)

# Save the clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")
print("Clinical data preview:")
print(preview_df(selected_clinical_df))

# 3. Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")

# Handle missing values
linked_data = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# 4. Check for bias in trait and demographic features
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Validate the data quality and save cohort info
note = "Dataset contains kidney tubulointerstitial gene expression data from patients with various forms of chronic kidney disease."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_biased,
    df=linked_data,
    note=note
)

# 6. Save the linked data if it's usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data not usable for the trait study - not saving final linked data.")